In [1]:
import pandas as pd
import numpy as np

from PIL import Image
from IPython.display import display, HTML, clear_output
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import sys
import os
DIR = os.getcwd()
import torch
import torchvision

from facebook_hateful_memes_detector.utils.globals import set_global, get_global
set_global("cache_dir", "/home/ahemf/cache/cache")
set_global("dataloader_workers", 8)
set_global("use_autocast", True)
set_global("models_dir", "/home/ahemf/cache/")

from facebook_hateful_memes_detector.preprocessing import DefinedRotation, QuadrantCut, DefinedAffine, DefinedColorJitter, DefinedRandomPerspective, ImageAugment
from PIL import Image
from facebook_hateful_memes_detector.utils import get_image_info_fn, set_device, get_device
from torchvision import transforms
import joblib
from tqdm.auto import tqdm, trange
from joblib import Parallel, delayed
from facebook_hateful_memes_detector.preprocessing import TextImageDataset, get_datasets, get_image2torchvision_transforms, TextAugment, get_transforms_for_bbox_methods
from facebook_hateful_memes_detector.models.external.detr import get_detr_model
from facebook_hateful_memes_detector.training import *

def hash(x):
    return joblib.hashing.hash(x, 'sha1')

def print_code(func):
    import inspect
    from pygments import highlight
    from pygments.lexers import PythonLexer
    from pygments.formatters import TerminalFormatter

    code = "".join(inspect.getsourcelines(func)[0])
    print(highlight(code, PythonLexer(), TerminalFormatter()))

set_device('cuda')



In [2]:
!wget https://raw.githubusercontent.com/airsplay/py-bottom-up-attention/master/demo/data/images/input.jpg


--2020-10-10 07:07:05--  https://raw.githubusercontent.com/airsplay/py-bottom-up-attention/master/demo/data/images/input.jpg
Resolving raw.githubusercontent.com... 151.101.52.133
Connecting to raw.githubusercontent.com|151.101.52.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209222 (204K) [image/jpeg]
Saving to: “input.jpg.1”

100%[======================================>] 209,222     --.-K/s   in 0.03s   

2020-10-10 07:07:06 (6.31 MB/s) - “input.jpg.1” saved [209222/209222]



In [2]:
data = get_datasets(data_dir="../data/",
                    train_text_transform=None,
                    train_image_transform=None,
                    test_text_transform=None,
                    test_image_transform=None,
                    train_torchvision_pre_image_transform=None,
                    test_torchvision_pre_image_transform=None,
                    cache_images=False,
                    use_images=True,
                    dev=False,
                    test_dev=True,
                    keep_original_text=True,
                    keep_original_image=True,
                    keep_processed_image=True,
                    keep_torchvision_image=False,
                    train_mixup_config=None)

data["test"]["label"] = -1

df = pd.concat((data["train"],
                data["dev"], 
                data["test"]))

dataset = convert_dataframe_to_dataset(df, data["metadata"], True)


In [ ]:
from time import sleep
for transformation in list(get_transforms_for_bbox_methods().transforms + get_transforms_for_multiview()):
    display(transformation(dataset[0]['original_image']))
    sleep(2)

In [ ]:
from facebook_hateful_memes_detector.utils import get_image_info_fn
from facebook_hateful_memes_detector.preprocessing import get_transforms_for_bbox_methods
from facebook_hateful_memes_detector.preprocessing import get_transforms_for_multiview
all_transforms = list(get_transforms_for_bbox_methods().transforms + get_transforms_for_multiview())
get_img_details = get_image_info_fn(enable_encoder_feats=False, device=get_device())["get_img_details"]
get_lxmert_details = get_image_info_fn(enable_encoder_feats=False, device=get_device())["get_lxmert_details"]

def lxmert_faster_rcnn_fn(img):
    _ = get_img_details(img)
    _ = get_lxmert_details(img)

for elem in tqdm(iter(dataset), total=len(dataset)):
    image = elem["original_image"]
    for idx, transformation in enumerate(all_transforms):
        lxmert_faster_rcnn_fn(transformation(image.copy()))
        if idx == 0:
            for _ in range(2):
                lxmert_faster_rcnn_fn(transformation(image.copy()))
        if idx == 2:
            for _ in range(8):
                lxmert_faster_rcnn_fn(transformation(image.copy()))
        
        


Config '/local/home/ahemf/mygit/facebook-hateful-memes/facebook_hateful_memes_detector/utils/faster_rcnn_R_101_C4_attr_caffemaxpool.yaml' has no VERSION. Assuming it to be compatible with latest v2.
Config '/local/home/ahemf/mygit/facebook-hateful-memes/facebook_hateful_memes_detector/utils/faster_rcnn_R_101_C4_attr_caffemaxpool.yaml' has no VERSION. Assuming it to be compatible with latest v2.


FeatureExtractor : Loaded Model...


/local/home/ahemf/mygit/vqa-maskrcnn-benchmark/maskrcnn_benchmark/structures/boxlist_ops.py:45: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:773.)
  keep = ((ws >= min_size) & (hs >= min_size)).nonzero().squeeze(1)


Modifications for VG in RPN (modeling/proposal_generator/rpn.py):
	Use hidden dim 512 instead fo the same dim as Res4 (1024).

Modifications for VG in RoI heads (modeling/roi_heads/fast_rcnn.py))
	Embedding: 1601 --> 256	Linear: 2304 --> 512	Linear: 512 --> 401

LXMERTFeatureExtractor : Loaded Model...


In [ ]:
cache_stats = get_global("cache_stats")
cache_stats['get_img_details']
cache_stats['get_lxmert_details']


In [2]:
img = Image.open("input.jpg")
im_transform = get_transforms_for_bbox_methods()
from facebook_hateful_memes_detector.preprocessing import HalfSwap, QuadrantCut, DefinedRotation, DefinedAffine
# im_transform = transforms.RandomAffine(0, scale=(1.25, 1.25))
hashes = Parallel(n_jobs=8, backend='threading')(delayed(lambda i: hash(im_transform(i)))(img.copy()) for i in trange(1000))
hashes = set(hashes)
len(hashes)




13

In [107]:
im = Image.open("input.jpg")

In [108]:
%timeit im_transform(im)

1.04 ms ± 27.1 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [ ]:
fns = get_image_info_fn(enable_encoder_feats=True, enable_image_captions=False)
cache_fns = [fns["get_img_details"], fns["get_encoder_feats"], 
             fns["get_lxmert_details"], 
             get_detr_model(get_device(), "detr_resnet50")["detr_fn"], 
             get_detr_model(get_device(), "detr_resnet50_panoptic")["detr_fn"], lambda x: x]

images = list(data["train"].img.values) + list(data["test"].img.values)
try:
    from torch.cuda.amp import GradScaler, autocast
    scaler = GradScaler()
    use_autocast = "cuda" in str(get_device())
except:
    pass
for i in tqdm(images):
    img = Image.open(i)
    for k, aug in augs_dict.items():
        for _ in range(16):
            img_copy = aug(img.copy())
            for fn in cache_fns:
                with autocast():
                    _ = fn(img_copy)
    



# We need Qcut for robustness
# We need DefinedAffine translate since in this problem we care about presence of object not position
# We need Image models trained on classification which only care about presence not position


Config '/local/home/ahemf/mygit/facebook-hateful-memes/facebook_hateful_memes_detector/utils/faster_rcnn_R_101_C4_attr_caffemaxpool.yaml' has no VERSION. Assuming it to be compatible with latest v2.


FeatureExtractor : Loaded Model...


/local/home/ahemf/mygit/vqa-maskrcnn-benchmark/maskrcnn_benchmark/structures/boxlist_ops.py:45: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  keep = ((ws >= min_size) & (hs >= min_size)).nonzero().squeeze(1)


ImageCaptionFeatures : Loaded Model...
LXMERTFeatureExtractor : Loaded Model...
DETR detr_resnet50 : Loaded Model...


Using cache found in /home/ahemf/.cache/torch/hub/facebookresearch_detr_master


DETR detr_resnet50_panoptic : Loaded Model...


Using cache found in /home/ahemf/.cache/torch/hub/facebookresearch_detr_master
IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
from transformers import AutoModelWithLMHead, AutoTokenizer, AutoModel
tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")
texts = pd.read_csv("text.csv", header=None)[0].values

m = lambda x: tokenizer.encode_plus(x, add_special_tokens=True, pad_to_max_length=False, truncation=False)
tlens = [len(d['input_ids']) for d in map(m, texts)]

np.percentile(tlens, [97, 99, 99.5, 99.9, 100])


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
set_device(device)

from facebook_hateful_memes_detector.utils import get_image_info_fn



In [ ]:
torchvision.transforms.RandomPerspective(p=1.0)(Image.open("input.jpg"))

In [3]:
fns = get_image_info_fn(enable_encoder_feats=True, enable_image_captions=True)


Config '/local/home/ahemf/mygit/facebook-hateful-memes/facebook_hateful_memes_detector/utils/faster_rcnn_R_101_C4_attr_caffemaxpool.yaml' has no VERSION. Assuming it to be compatible with latest v2.


In [2]:
from facebook_hateful_memes_detector.utils.detectron_v1_object_detector import LXMERTFeatureExtractor, persistent_caching_fn
lxmert_feature_extractor = LXMERTFeatureExtractor(get_device(), do_autocast=False)
def fn(x):
    return x
fn = persistent_caching_fn(fn, "random_2323", False)
fn(2)
img = Image.open("input.jpg")
from torch.cuda.amp import autocast

feats = lxmert_feature_extractor(img)  

feats[0].scores.mean()
feats[1].mean()
feats[0].pred_boxes
feats[0].scores









Config '/local/home/ahemf/mygit/facebook-hateful-memes/facebook_hateful_memes_detector/utils/faster_rcnn_R_101_C4_attr_caffemaxpool.yaml' has no VERSION. Assuming it to be compatible with latest v2.


2

Modifications for VG in RPN (modeling/proposal_generator/rpn.py):
	Use hidden dim 512 instead fo the same dim as Res4 (1024).

Modifications for VG in RoI heads (modeling/roi_heads/fast_rcnn.py))
	Embedding: 1601 --> 256	Linear: 2304 --> 512	Linear: 512 --> 401

LXMERTFeatureExtractor : Loaded Model...


/home/ahemf/anaconda3/lib/python3.7/site-packages/detectron2/modeling/roi_heads/fast_rcnn.py:101: UserWarning: This overload of nonzero is deprecated:
	nonzero()
Consider using one of the following signatures instead:
	nonzero(*, bool as_tuple) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:773.)
  filter_inds = filter_mask.nonzero()


tensor(0.4331, device='cuda:0')

tensor(0.5820, device='cuda:0')

Boxes(tensor([[169.3243, 214.3307, 485.1099, 469.7549],
        [100.7880, 251.3626, 394.6440, 480.0000],
        [586.3296,   0.0000, 638.9190, 353.0089],
        [608.1302,  61.5700, 637.4992, 423.6490],
        [281.5395, 162.0848, 328.3549, 194.3590],
        [157.3794,  92.9709, 392.1783, 461.2858],
        [222.4158, 291.6465, 566.1779, 480.0000],
        [ 41.3324, 181.5984, 349.9705, 450.4261],
        [ 13.8716,   1.7292, 207.8326, 437.1108],
        [381.5179, 310.5241, 618.9825, 417.9209],
        [261.5557, 192.7572, 362.9944, 429.1843],
        [203.8108,   0.0000, 300.6809, 279.9798],
        [420.2803, 278.9125, 637.1599, 389.4050],
        [  2.9243,   0.0000, 123.0734, 353.0846],
        [ 55.2990,  27.5763, 128.4013, 386.0382],
        [246.5886, 159.6546, 347.5096, 360.4674],
        [ 44.2803, 321.1058, 584.3936, 477.0761],
        [171.0541, 371.0668, 462.8420, 469.1741],
        [101.0942, 165.9819, 550.5664, 416.4953],
        [  1.5654,   0.0000, 529.4667, 280.5

tensor([0.9688, 0.9398, 0.7171, 0.7034, 0.6957, 0.6017, 0.5914, 0.5399, 0.5305,
        0.5088, 0.4935, 0.4671, 0.4599, 0.4557, 0.4474, 0.4470, 0.4280, 0.3849,
        0.3726, 0.3425, 0.3416, 0.3362, 0.3179, 0.3128, 0.3056, 0.2993, 0.2967,
        0.2910, 0.2739, 0.2670, 0.2654, 0.2511, 0.2486, 0.2417, 0.2259, 0.2201],
       device='cuda:0')

In [17]:
from facebook_hateful_memes_detector.utils.detectron_v1_object_detector import LXMERTFeatureExtractor
lxmert_feature_extractor = LXMERTFeatureExtractor(get_device())
img = Image.open("input.jpg")
from torch.cuda.amp import autocast
with autocast():
    feats = lxmert_feature_extractor(img)  
len(feats)
feats[0].scores.mean()
feats[1].mean()
feats[0].pred_boxes
feats[0].scores









Config '/local/home/ahemf/mygit/facebook-hateful-memes/facebook_hateful_memes_detector/utils/faster_rcnn_R_101_C4_attr_caffemaxpool.yaml' has no VERSION. Assuming it to be compatible with latest v2.


2

tensor(0.4330, device='cuda:0')

tensor(0.5820, device='cuda:0', dtype=torch.float16)

Boxes(tensor([[1.6930e+02, 2.1420e+02, 4.8495e+02, 4.6980e+02],
        [1.0069e+02, 2.5155e+02, 3.9438e+02, 4.8000e+02],
        [5.8632e+02, 0.0000e+00, 6.3880e+02, 3.5310e+02],
        [6.0791e+02, 6.1763e+01, 6.3700e+02, 4.2360e+02],
        [2.8146e+02, 1.6200e+02, 3.2840e+02, 1.9440e+02],
        [1.5730e+02, 9.3000e+01, 3.9198e+02, 4.6140e+02],
        [2.2253e+02, 2.9175e+02, 5.6622e+02, 4.8000e+02],
        [4.1425e+01, 1.8150e+02, 3.5029e+02, 4.5030e+02],
        [1.3899e+01, 1.6840e+00, 2.0769e+02, 4.3710e+02],
        [3.7998e+02, 3.1110e+02, 6.2021e+02, 4.1670e+02],
        [2.6152e+02, 1.9245e+02, 3.6289e+02, 4.2900e+02],
        [2.0379e+02, 0.0000e+00, 3.0066e+02, 2.7990e+02],
        [4.2017e+02, 2.7855e+02, 6.3700e+02, 3.8910e+02],
        [2.9381e+00, 0.0000e+00, 1.2311e+02, 3.5310e+02],
        [5.5258e+01, 2.7713e+01, 1.2843e+02, 3.8610e+02],
        [2.4652e+02, 1.5975e+02, 3.4759e+02, 3.6060e+02],
        [4.4124e+01, 3.2100e+02, 5.8452e+02, 4.7700e+02],
        

tensor([0.9688, 0.9399, 0.7174, 0.7038, 0.6960, 0.6010, 0.5914, 0.5405, 0.5300,
        0.4952, 0.4940, 0.4683, 0.4603, 0.4561, 0.4477, 0.4462, 0.4271, 0.3841,
        0.3730, 0.3433, 0.3431, 0.3367, 0.3188, 0.3125, 0.3050, 0.2994, 0.2975,
        0.2961, 0.2740, 0.2673, 0.2652, 0.2508, 0.2483, 0.2417, 0.2267, 0.2204],
       device='cuda:0')

In [ ]:
res, info = fns["feature_extractor"](Image.open("../data/img/08291.png"))
res[:2, :8]
info["boxes"][:4]
info["objects"]


In [ ]:
res, info = fns["get_img_details"](Image.open("../data/img/08291.png"))
res[:2, :8]
info["boxes"][:4]
info["objects"]


In [ ]:
fns["get_image_captions"]("../data/img/08291.png")

In [ ]:
im = im_transform(Image.open("input.jpg"))
im

In [ ]:
instances, roi_features = fns["get_lxmert_details"](im)
instances.pred_boxes.tensor # boxes
roi_features # feats
# (feats, boxes)

In [ ]:
fns["get_encoder_feats"]("../data/img/08291.png")
